In [1]:
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format

In [3]:
df = pd.read_csv('../../data/processed/combined data/combined.csv')

In [4]:
df_cleaned = df.copy()

# Precess the data like Dris 

In [5]:
relevant_features = [
    'startingAirport',          # Categorical
    'destinationAirport',       # Categorical
    'travelDuration',           # Duration as time string, needs conversion
    'totalTravelDistance',      # Numerical
    'isNonStop',                # Binary
    'isBasicEconomy',           # Binary
    'segmentsDepartureTimeRaw',
    'segmentsCabinCode',        # Categorical for cabin class
    'segmentsAirlineName',      # Categorical
    'searchDate',               # Date, needs feature extraction
    'flightDate',               # Date, needs feature extraction
    'isRefundable',             # Binary
    'totalFare'                 # Target variable
]

In [6]:
df_cleaned = df_cleaned[relevant_features]
df_cleaned

,startingAirport,destinationAirport,travelDuration,totalTravelDistance,isNonStop,isBasicEconomy,segmentsDepartureTimeRaw,segmentsCabinCode,segmentsAirlineName,searchDate,flightDate,isRefundable,totalFare
0,OAK,ATL,PT6H38M,2281.00,False,False,2022-05-13T11:42:00.000-07:00||2022-05-13T14:0...,coach||coach,Delta||Delta,2022-05-11,2022-05-13,False,317.60
1,OAK,ATL,PT6H17M,NaN,False,False,2022-05-13T20:56:00.000-07:00||2022-05-13T23:2...,coach||coach,Spirit Airlines||Spirit Airlines,2022-05-11,2022-05-13,False,379.58
2,OAK,ATL,PT6H15M,NaN,False,False,2022-05-13T08:56:00.000-07:00||2022-05-13T10:5...,coach||coach,Spirit Airlines||Spirit Airlines,2022-05-11,2022-05-13,False,463.58
3,OAK,ATL,PT9H6M,NaN,False,False,2022-05-13T06:05:00.000-07:00||2022-05-13T10:5...,coach||coach,Spirit Airlines||Spirit Airlines,2022-05-11,2022-05-13,False,463.58
4,OAK,ATL,PT14H13M,NaN,False,False,2022-05-13T13:00:00.000-07:00||2022-05-13T23:2...,coach||coach,Spirit Airlines||Spirit Airlines,2022-05-11,2022-05-13,False,463.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13519994,MIA,SFO,PT9H58M,3039.00,False,False,2022-05-20T12:32:00.000-04:00||2022-05-20T16:5...,coach||coach,United||Alaska Airlines,2022-05-05,2022-05-20,False,770.60
13519995,MIA,SFO,PT11H40M,3670.00,False,False,2022-05-20T07:50:00.000-04:00||2022-05-20T13:0...,coach||coach,Delta||Alaska Airlines,2022-05-05,2022-05-20,False,801.60
13519996,MIA,SFO,PT11H14M,3670.00,False,False,2022-05-20T14:53:00.000-04:00||2022-05-20T19:4...,coach||coach,United||Alaska Airlines,2022-05-05,2022-05-20,False,813.60
13519997,MIA,SFO,PT11H24M,3670.00,False,False,2022-05-20T11:57:00.000-04:00||2022-05-20T17:0...,coach||coach,United||Alaska Airlines,2022-05-05,2022-05-20,False,813.60


In [7]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13519999 entries, 0 to 13519998
Data columns (total 13 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   startingAirport           object 
 1   destinationAirport        object 
 2   travelDuration            object 
 3   totalTravelDistance       float64
 4   isNonStop                 bool   
 5   isBasicEconomy            bool   
 6   segmentsDepartureTimeRaw  object 
 7   segmentsCabinCode         object 
 8   segmentsAirlineName       object 
 9   searchDate                object 
 10  flightDate                object 
 11  isRefundable              bool   
 12  totalFare                 float64
dtypes: bool(3), float64(2), object(8)
memory usage: 1.0+ GB


In [8]:
# Function to convert duration from format 'PT#H#M' to total hours in float
def convert_duration_to_hours(duration):
    # Extract hours and minutes using regex
    hours = re.search(r'(\d+)H', duration)
    minutes = re.search(r'(\d+)M', duration)
    
    # Convert extracted values to integers, defaulting to 0 if not found
    hours = int(hours.group(1)) if hours else 0
    minutes = int(minutes.group(1)) if minutes else 0
    
    # Calculate total hours
    total_hours = hours + minutes / 60.0
    return total_hours

# Applying the conversion to the travelDuration column
df_cleaned['travelDurationHours'] = df_cleaned['travelDuration'].apply(convert_duration_to_hours)

# Output the first few rows to verify
df_cleaned[['travelDurationHours']].head()

,travelDurationHours
0,6.63
1,6.28
2,6.25
3,9.10
4,14.22


In [9]:
df_cleaned = df_cleaned.drop(columns=['travelDuration'])
df_cleaned.head()

,startingAirport,destinationAirport,totalTravelDistance,isNonStop,isBasicEconomy,segmentsDepartureTimeRaw,segmentsCabinCode,segmentsAirlineName,searchDate,flightDate,isRefundable,totalFare,travelDurationHours
0,OAK,ATL,2281.00,False,False,2022-05-13T11:42:00.000-07:00||2022-05-13T14:0...,coach||coach,Delta||Delta,2022-05-11,2022-05-13,False,317.60,6.63
1,OAK,ATL,NaN,False,False,2022-05-13T20:56:00.000-07:00||2022-05-13T23:2...,coach||coach,Spirit Airlines||Spirit Airlines,2022-05-11,2022-05-13,False,379.58,6.28
2,OAK,ATL,NaN,False,False,2022-05-13T08:56:00.000-07:00||2022-05-13T10:5...,coach||coach,Spirit Airlines||Spirit Airlines,2022-05-11,2022-05-13,False,463.58,6.25
3,OAK,ATL,NaN,False,False,2022-05-13T06:05:00.000-07:00||2022-05-13T10:5...,coach||coach,Spirit Airlines||Spirit Airlines,2022-05-11,2022-05-13,False,463.58,9.10
4,OAK,ATL,NaN,False,False,2022-05-13T13:00:00.000-07:00||2022-05-13T23:2...,coach||coach,Spirit Airlines||Spirit Airlines,2022-05-11,2022-05-13,False,463.58,14.22


In [10]:
# Function to count the number of segments in the 'segmentsCabinCode' column
def count_segments(cabin_code):
    # Count '||' to get the number of segments, adding 1 for the total count
    return cabin_code.count('||') + 1

# Applying the function to create the new 'numSegments' feature
df_cleaned['numSegments'] = df_cleaned['segmentsCabinCode'].apply(count_segments)

# Output first few rows to verify the new feature
df_cleaned[['segmentsCabinCode', 'numSegments']].head()

,segmentsCabinCode,numSegments
0,coach||coach,2
1,coach||coach,2
2,coach||coach,2
3,coach||coach,2
4,coach||coach,2


In [12]:
# Function to create a binary feature indicating if there are mixed cabin classes across segments
def has_mixed_cabin_classes(cabin_code):
    # Split by '||' and check if all elements are the same
    cabins = cabin_code.split('||')
    return 1 if len(set(cabins)) > 1 else 0

# Applying the function to create the new 'mixedCabinClasses' feature
df_cleaned['mixedCabinClasses'] = df_cleaned['segmentsCabinCode'].apply(has_mixed_cabin_classes)

# Function to create a binary feature indicating if multiple airlines are involved in the flight
def has_multiple_airlines(airline_name):
    # Split by '||' and check if all elements are the same
    airlines = airline_name.split('||')
    return 1 if len(set(airlines)) > 1 else 0

# Applying the function to create the new 'multipleAirlines' feature
df_cleaned['multipleAirlines'] = df_cleaned['segmentsAirlineName'].apply(has_multiple_airlines)

# Displaying the new features in the first 10 rows to verify
df_cleaned[['segmentsCabinCode', 'mixedCabinClasses', 'segmentsAirlineName', 'multipleAirlines']].head(10)

,segmentsCabinCode,mixedCabinClasses,segmentsAirlineName,multipleAirlines
0,coach||coach,0,Delta||Delta,0
1,coach||coach,0,Spirit Airlines||Spirit Airlines,0
2,coach||coach,0,Spirit Airlines||Spirit Airlines,0
3,coach||coach,0,Spirit Airlines||Spirit Airlines,0
4,coach||coach,0,Spirit Airlines||Spirit Airlines,0
5,coach||coach,0,Delta||Delta,0
6,coach||coach||coach,0,Spirit Airlines||Spirit Airlines||Spirit Airlines,0
7,coach||coach||coach,0,Spirit Airlines||Spirit Airlines||Spirit Airlines,0
8,coach||coach||coach,0,Delta||United||United,1
9,coach||coach||coach,0,Delta||United||United,1


In [15]:
# Function to categorize cabin codes based on similarity or mixed types
def categorize_cabin_code(cabin_code):
    # Split the cabin_code by '||' to get individual cabin types
    cabins = cabin_code.split('||')
    # Check if all cabin types are the same, return that cabin type if true; otherwise, return 'mixed'
    return cabins[0] if len(set(cabins)) == 1 else 'mixed'

# Applying the function to create a categorized 'cabinType' feature
df_cleaned['cabinType'] = df_cleaned['segmentsCabinCode'].apply(categorize_cabin_code)

# Displaying the first few rows to verify the new 'cabinType' feature
df_cleaned[['segmentsCabinCode', 'cabinType']].head(10)


,segmentsCabinCode,cabinType
0,coach||coach,coach
1,coach||coach,coach
2,coach||coach,coach
3,coach||coach,coach
4,coach||coach,coach
5,coach||coach,coach
6,coach||coach||coach,coach
7,coach||coach||coach,coach
8,coach||coach||coach,coach
9,coach||coach||coach,coach


In [18]:
# Dropping the 'segmentsCabinCode' and 'segmentsAirlineName' features
df_cleaned = df_cleaned.drop(columns=['segmentsCabinCode', 'segmentsAirlineName'])

In [19]:
df_cleaned.head()

,startingAirport,destinationAirport,totalTravelDistance,isNonStop,isBasicEconomy,segmentsDepartureTimeRaw,searchDate,flightDate,isRefundable,totalFare,travelDurationHours,numSegments,mixedCabinClasses,multipleAirlines,cabinType
0,OAK,ATL,2281.00,False,False,2022-05-13T11:42:00.000-07:00||2022-05-13T14:0...,2022-05-11,2022-05-13,False,317.60,6.63,2,0,0,coach
1,OAK,ATL,NaN,False,False,2022-05-13T20:56:00.000-07:00||2022-05-13T23:2...,2022-05-11,2022-05-13,False,379.58,6.28,2,0,0,coach
2,OAK,ATL,NaN,False,False,2022-05-13T08:56:00.000-07:00||2022-05-13T10:5...,2022-05-11,2022-05-13,False,463.58,6.25,2,0,0,coach
3,OAK,ATL,NaN,False,False,2022-05-13T06:05:00.000-07:00||2022-05-13T10:5...,2022-05-11,2022-05-13,False,463.58,9.10,2,0,0,coach
4,OAK,ATL,NaN,False,False,2022-05-13T13:00:00.000-07:00||2022-05-13T23:2...,2022-05-11,2022-05-13,False,463.58,14.22,2,0,0,coach


In [20]:
# Convert 'searchDate' and 'flightDate' to datetime format
df_cleaned['searchDate'] = pd.to_datetime(df_cleaned['searchDate'])
df_cleaned['flightDate'] = pd.to_datetime(df_cleaned['flightDate'])

# Calculate the days until departure
df_cleaned['daysUntilDeparture'] = (df_cleaned['flightDate'] - df_cleaned['searchDate']).dt.days

# Display the new feature along with search and flight dates to verify
print(df_cleaned[['searchDate', 'flightDate', 'daysUntilDeparture']].head(10))

  searchDate flightDate  daysUntilDeparture
0 2022-05-11 2022-05-13                   2
1 2022-05-11 2022-05-13                   2
2 2022-05-11 2022-05-13                   2
3 2022-05-11 2022-05-13                   2
4 2022-05-11 2022-05-13                   2
5 2022-05-11 2022-05-13                   2
6 2022-05-11 2022-05-13                   2
7 2022-05-11 2022-05-13                   2
8 2022-05-11 2022-05-13                   2
9 2022-05-11 2022-05-13                   2


In [21]:
df_cleaned = df_cleaned.drop(columns=['searchDate', 'flightDate'])

In [22]:
# Check for missing data in the cleaned dataset
missing_data_summary = df_cleaned.isnull().sum()

# Display columns with missing values only
print(missing_data_summary[missing_data_summary > 0])

totalTravelDistance    959619
dtype: int64


In [23]:
# Removing rows with missing values in 'totalTravelDistance'
df_cleaned = df_cleaned.dropna(subset=['totalTravelDistance'])

# Confirming that there are no more missing values
print("Missing values in 'totalTravelDistance' after removal:", df_cleaned['totalTravelDistance'].isnull().sum())

Missing values in 'totalTravelDistance' after removal: 0


In [24]:
# Function to extract the hour and classify it into a time block
def get_time_block(departure_time_raw):
    # Extract the first time in the series (before '||')
    first_departure_time = departure_time_raw.split('||')[0]
    # Extract hour from the first time string using regex
    match = re.search(r'T(\d{2}):', first_departure_time)
    if match:
        hour = int(match.group(1))
        # Classify hour into time blocks
        if 5 <= hour < 12:
            return 'morning'
        elif 12 <= hour < 17:
            return 'afternoon'
        elif 17 <= hour < 21:
            return 'evening'
        else:
            return 'night'
    return 'unknown'  # For any format issues or missing data

# Apply the function to create a new 'timeBlock' column
df_cleaned['timeBlock'] = df_cleaned['segmentsDepartureTimeRaw'].apply(get_time_block)

# Displaying the result
df_cleaned[['segmentsDepartureTimeRaw', 'timeBlock']].head()

,segmentsDepartureTimeRaw,timeBlock
0,2022-05-13T11:42:00.000-07:00||2022-05-13T14:0...,morning
5,2022-05-13T17:18:00.000-07:00||2022-05-13T23:4...,evening
8,2022-05-13T07:16:00.000-07:00||2022-05-13T11:2...,morning
9,2022-05-13T07:16:00.000-07:00||2022-05-13T12:5...,morning
10,2022-05-13T18:58:00.000-07:00||2022-05-14T00:5...,evening


In [25]:
df_cleaned['timeBlock'].value_counts()

timeBlock
morning      6191965
afternoon    3350400
evening      1935114
night        1082901
Name: count, dtype: int64

In [27]:
df_cleaned = df_cleaned.drop(columns=['segmentsDepartureTimeRaw'])

In [32]:
df = df_cleaned.copy()

In [33]:
from sklearn.preprocessing import LabelEncoder

# Combine unique values from both columns to ensure consistent encoding
all_airports = pd.concat([df['startingAirport'], df['destinationAirport']]).unique()

# Initialize and fit LabelEncoder with combined unique values
airport_encoder = LabelEncoder()
airport_encoder.fit(all_airports)

# Apply the encoder to both 'startingAirport' and 'destinationAirport'
df['startingAirport_encoded'] = airport_encoder.transform(df['startingAirport'])
df['destinationAirport_encoded'] = airport_encoder.transform(df['destinationAirport'])

# Verify encoding matches by printing first few rows of encoded columns
print(df[['startingAirport', 'startingAirport_encoded', 'destinationAirport', 'destinationAirport_encoded']].head())

   startingAirport  startingAirport_encoded destinationAirport  \
0              OAK                       12                ATL   
5              OAK                       12                ATL   
8              OAK                       12                ATL   
9              OAK                       12                ATL   
10             OAK                       12                ATL   

    destinationAirport_encoded  
0                            0  
5                            0  
8                            0  
9                            0  
10                           0  


In [34]:
# Applying label encoding for boolean features
df['isNonStop_encoded'] = df['isNonStop'].astype(int)
df['isBasicEconomy_encoded'] = df['isBasicEconomy'].astype(int)
df['isRefundable_encoded'] = df['isRefundable'].astype(int)

In [35]:
df['isNonStop_encoded'].value_counts()

isNonStop_encoded
0    8731430
1    3828950
Name: count, dtype: int64

In [36]:
# Initialize LabelEncoder for 'cabinType'
cabin_type_encoder = LabelEncoder()
df['cabinType_encoded'] = cabin_type_encoder.fit_transform(df['cabinType'])

# Verify the encoding by displaying the first few rows
df[['cabinType', 'cabinType_encoded']].head()

,cabinType,cabinType_encoded
0,coach,1
5,coach,1
8,coach,1
9,coach,1
10,coach,1


In [38]:
# Define a custom mapping for time blocks
time_block_mapping = {'morning': 0, 'afternoon': 1, 'evening': 2, 'night': 3}

# Apply the mapping to create the encoded 'timeBlock_encoded' column
df['timeBlock_encoded'] = df['timeBlock'].map(time_block_mapping)

# Display the encoded values to verify
print(df[['timeBlock', 'timeBlock_encoded']].head())

   timeBlock  timeBlock_encoded
0    morning                  0
5    evening                  2
8    morning                  0
9    morning                  0
10   evening                  2


In [39]:
# Dropping categorical features
df_num = df.drop(columns=['startingAirport', 'destinationAirport','isNonStop','isBasicEconomy','isRefundable','cabinType','timeBlock'])

In [40]:
df_num = df_num.drop(columns=['mixedCabinClasses','multipleAirlines'])

In [41]:
df_num.head()

,totalTravelDistance,totalFare,travelDurationHours,numSegments,daysUntilDeparture,startingAirport_encoded,destinationAirport_encoded,isNonStop_encoded,isBasicEconomy_encoded,isRefundable_encoded,cabinType_encoded,timeBlock_encoded
0,2281.00,317.60,6.63,2,2,12,0,0,0,0,1,0
5,2175.00,487.60,9.05,2,2,12,0,0,0,0,1,2
8,2393.00,526.11,12.48,3,2,12,0,0,0,0,1,0
9,2393.00,531.71,12.48,3,2,12,0,0,0,0,1,0
10,2150.00,534.01,7.87,2,2,12,0,0,0,0,1,2


In [42]:
df_num.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12560380 entries, 0 to 13519998
Data columns (total 12 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   totalTravelDistance         float64
 1   totalFare                   float64
 2   travelDurationHours         float64
 3   numSegments                 int64  
 4   daysUntilDeparture          int64  
 5   startingAirport_encoded     int64  
 6   destinationAirport_encoded  int64  
 7   isNonStop_encoded           int64  
 8   isBasicEconomy_encoded      int64  
 9   isRefundable_encoded        int64  
 10  cabinType_encoded           int64  
 11  timeBlock_encoded           int64  
dtypes: float64(3), int64(9)
memory usage: 1.2 GB


In [43]:
df_num

,totalTravelDistance,totalFare,travelDurationHours,numSegments,daysUntilDeparture,startingAirport_encoded,destinationAirport_encoded,isNonStop_encoded,isBasicEconomy_encoded,isRefundable_encoded,cabinType_encoded,timeBlock_encoded
0,2281.00,317.60,6.63,2,2,12,0,0,0,0,1,0
5,2175.00,487.60,9.05,2,2,12,0,0,0,0,1,2
8,2393.00,526.11,12.48,3,2,12,0,0,0,0,1,0
9,2393.00,531.71,12.48,3,2,12,0,0,0,0,1,0
10,2150.00,534.01,7.87,2,2,12,0,0,0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...
13519994,3039.00,770.60,9.97,2,15,11,15,0,0,0,1,1
13519995,3670.00,801.60,11.67,2,15,11,15,0,0,0,1,0
13519996,3670.00,813.60,11.23,2,15,11,15,0,0,0,1,1
13519997,3670.00,813.60,11.40,2,15,11,15,0,0,0,1,0


In [44]:
output_path = '../../data/processed/df_prep.csv'
df_num.to_csv(output_path, index=False)

print("Data saved to:", output_path)

Data saved to: ../../data/processed/df_prep.csv
